<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dados-Fluviométricos" data-toc-modified-id="Dados-Fluviométricos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dados Fluviométricos</a></span><ul class="toc-item"><li><span><a href="#Long-term-flow-duration" data-toc-modified-id="Long-term-flow-duration-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Long term flow duration</a></span></li><li><span><a href="#Decomposição-de-ts" data-toc-modified-id="Decomposição-de-ts-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Decomposição de ts</a></span></li></ul></li><li><span><a href="#Cumsum-running-mean" data-toc-modified-id="Cumsum-running-mean-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Cumsum running mean</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
pyo.init_notebook_mode(connected=True)

- Gerando base de dados e leitura dos dados

In [2]:
%run gerar_ribeiroGoncalves.py

/home/breno/temp/hidropower-parnaiba/gerar_ribeiroGoncalves.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [84]:
ts = pd.read_csv('ribeiroGon.csv', index_col=0).drop_duplicates(subset=['month', 'year'])

- Utilizando somente os dados dos ultimos 30 anos

In [85]:
ts = ts[ts['year'] >= (max(ts['year']) - 30)]

In [86]:
ts['data_ticks'] = ts['year'].astype('str')+'-'+ts['month'].astype('str')+'-01'
ts['data_ticks'] = pd.to_datetime(ts['data_ticks'])

# Dados Fluviométricos

In [87]:
def running_mean(x):
    # Série a ser iterada
    ts = range(len(x))[1:-1]
    # retornando lista com a média dos elementos vizinhos
    return [(x[i-1]+x[i+1])/2 for i in ts]

In [88]:
def running_mean2(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [89]:
ts.head()

,month,year,vazao,data_ticks
283,1,1988,274.334,1988-01-01
284,2,1988,300.320,1988-02-01
285,3,1988,367.603,1988-03-01
286,4,1988,282.685,1988-04-01
287,5,1988,213.099,1988-05-01


In [90]:
x = ts.sort_values(['year','month'])['data_ticks']
y = ts.sort_values(['year','month'])['vazao'].values
x_raw = list(range(len(x)))

In [91]:
trace = go.Scatter(x = x, y = y, name = 'Medias')

trace_moving_avg = go.Scatter(x = x, y = running_mean(y), name = 'Média móvel (neighbors)')

trace_moving_avg2 = go.Scatter(x = x[2:], y = running_mean2(y,4), name = 'Média móvel (cumsum)')

tendencia = go.Scatter()

layout = go.Layout(title = 'Medidas de vazão do Rio Parnaíba - Ribeiro Gonçalves',
                   xaxis = dict(title = 'Data'),
                   yaxis = dict(title = 'Vazão média (m³/s)'))

fig = go.Figure(data = [trace], layout = layout)#, trace_moving_avg, trace_moving_avg2

pyo.iplot(fig, 'vazoes.html', config = dict(showLink = False))

In [96]:
a = ts['vazao']

In [98]:
a.std()

73.39138780542282

In [80]:
temp_ts  = ts.groupby(['year']).mean()['vazao']

trace = go.Scatter(x = temp_ts.index, y = temp_ts.values, name = 'Medias')

trace_moving_avg = go.Scatter(x = x, y = running_mean(y), name = 'Média móvel (neighbors)')
tendencia = go.Scatter()

layout = go.Layout(title = 'Vazões médias anuais do Rio Parnaíba - Ribeiro Gonçalves',
                   xaxis = dict(title = 'Data'),
                   yaxis = dict(title = 'Vazão média (m³/s)'))

fig = go.Figure(data = [trace], layout = layout)#, trace_moving_avg, trace_moving_avg2

pyo.iplot(fig, 'vazoes.html', config = dict(showLink = False))

In [81]:
np.mean(ts[ts['year'] == 2018][['month','vazao']].groupby('month').mean()['vazao'].values)

173.3656

In [82]:
x = ['Jan','Fev','Mar','Abr', 'Maio','Jun','Jul','Ago','Set','Out','Nov','Dez']
trace = go.Bar(x = x,
                   y = ts[['month','vazao']].groupby('month').mean()['vazao'].values)

media_geral = np.mean(ts[['month','vazao']].groupby('month').mean()['vazao'].values)

trace_line = go.Scatter(x = x, y = [media_geral]*12, mode = 'lines')

layout = go.Layout(title = 'Médias mensais de vazão do Rio Parnaíba - Ribeiro Gonçalves', yaxis = dict(title = 'Vazão (m³/s)'),
                   xaxis = dict(title = 'Meses'), showlegend=False)
fig = go.Figure(data = [trace, trace_line], layout = layout)

pyo.iplot(fig, 'medias_mensais.html', config=dict(showLink = False))

In [58]:
media_geral

202.9225386200717

- Segundo a média: meses de seca = Maio a Novembro e meses de cheia = Dez a Abr

In [62]:
data = []
meses_list = ['Jan','Fev','Mar','Abr', 'Maio','Jun','Jul','Ago','Set','Out','Nov','Dez']
for mes in np.arange(1,13,1):
    data.append(go.Box(y = ts[ts['month'] == mes]['vazao'], name = meses_list[mes-1]))

layout = go.Layout(title = 'Boxplot mensal das medidas de vazão', showlegend = False)

fig = go.Figure(data = data, layout = layout)

pyo.iplot(fig, filename ='vazoes_medias_mensais.html', config=dict(showLink=False))

In [13]:
layout = go.Layout(title = 'Distribuição vazões', barmode = 'overlay')

fig = go.Figure(data = [go.Histogram(x = ts['vazao'], opacity = 0.75, name = 'Observado'),
                        go.Histogram(x = running_mean2(y,4), opacity = 0.75, name = 'Média movel')],
                        layout = layout)
pyo.iplot(fig, config=dict(showLink = False))

## Long term flow duration

In [110]:
cumsum = np.cumsum(ts['vazao'])
#[i for i in range(len(cumsum))][::-1]
x = 100*cumsum/np.max(cumsum)

trace = go.Scatter(y=np.sort(ts['vazao'])[::-1], x=x,
                     marker=dict(color='rgb(150, 25, 120)'), name = 'Vazão original', mode = 'markers')
trace2 = go.Scatter(x=x, y = np.sort(running_mean2(y,4))[::-1], name = 'Vazão com média móvel', mode = 'markers')
trace3 = go.Scatter(y = [203.01851219512204,203.01851219512204], x = [0,43.2],
                    marker = dict(color = 'gray'), name = 'Vazão média', mode = 'lines',
                    line = dict(dash = 'dot'))
trace3a = go.Scatter(y = [ts['vazao'].min(),203.01851219512204], x = [43.2,43.2],
                    marker = dict(color = 'gray'), name = 'Vazão média', mode = 'lines',
                    line = dict(dash = 'dot'))

trace4 = go.Scatter(y = [ts['vazao'].min(), 125.06], x = [95,95],
                    marker = dict(color = 'gray'), name = 'Q_95', mode = 'lines',
                    line = dict(dash = 'dot'))
trace4a = go.Scatter(y = [125.06, 125.06], x = [0,95],
                    marker = dict(color = 'gray'), name = 'Q_95', mode = 'lines',
                    line = dict(dash = 'dot'), showlegend = False)

layout = go.Layout(
    title="Curva de permanência do Rio Parnaíba - Ribeiro Gonçalves",
    xaxis = dict(title = 'frequencia de excedencia (%)'), yaxis = dict(title = 'Taxa de vazão Q (m^3/s)'),
    hovermode = 'closest'
)
fig = go.Figure(data=[trace,trace3, trace3a, trace4, trace4a], layout=layout)#trace2
pyo.plot(fig, filename='curva_permanencia.html', config=dict(showLink = False))

'file:///home/breno/temp/hidropower-parnaiba/curva_permanencia.html'

In [111]:
print('Vazão média:', ts['vazao'].mean())
print('Q95:', 125.06)
print('How flashy the watercourse is?', 125.06/ts['vazao'].mean())

Vazão média: 203.01851219512196
Q95: 125.06
How flashy the watercourse is? 0.6160029380956369


Segundo o link do Rodrigo, o nosso rio não tende a inundar. Ver o (link)[http://www.renewablesfirst.co.uk/hydropower/hydropower-learning-centre/what-is-a-flow-duration-curve/].

Último parágrafo com as infos:
>Flow rates between Q0 and Q10 are considered high flow rates, and Q0 to Q1 would be extreme flood events. It is important that hydropower systems are designed to cope with such extreme flows. Flows from Q10 to Q70 would be the ‘medium’ range of flows and you would want your hydropower system to operate efficiently right across these flow rates. Flow rates from Q70 to Q100 are the ‘low flows’ when hydropower systems will just be operating but at a low power output, and as you move further to the right on the FDC hydro systems will begin to shut down due to low flow. As flow rates move from Q95 towards Q100 you move into the low-flow draught flows.

## Decomposição de ts

# Cumsum running mean

https://www.dallasfed.org/research/basics/moving.aspx

https://stats.stackexchange.com/questions/144013/smoothing-when-to-use-it-and-when-not-to